In [44]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re


/Users/tammi/Desktop/CS 638 Stuffs/cs638project/learning


In [45]:
from numpy import genfromtxt
# train = pd.read_csv('train_reduced.csv', encoding="ISO-8859-1", index_col='_id')
# test = pd.read_csv('test_reduced.csv', encoding="ISO-8859-1", index_col='_id')
from numpy import genfromtxt
#myData = genfromtxt('train.csv',delimiter=',')

train = genfromtxt('train_reduced_CME_test.csv',delimiter=',')
test = genfromtxt('test_reduced_CME_test.csv',delimiter=',')

print(train.shape)
# test = test.dropna()
# test.target
train = train[~np.isnan(train).any(axis=1)]

y = train[:,4] # label
X = train[:,0:3] # data

(281, 5)


In [46]:
from sklearn.cross_validation import cross_val_score

# TODO use F-1 for scoring, or a hybrid if possible
# we can use a for loop ['f1', 'accuracy', ]
# http://scikit-learn.org/stable/modules/model_evaluation.html

# logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X,y)
print('Logistic Regression:', cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

# decision tree
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(X,y)
print('Decision Tree:', cross_val_score(clf, X, y, cv=10, scoring='accuracy').mean())

# random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=10, max_depth=10)
forest.fit(X,y)
print('Random Forest:', cross_val_score(forest, X, y, cv=10, scoring='accuracy').mean())

# support vector machine
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(X,y)
print('SVM:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# naive bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X,y)
print('Naive Bayes:', cross_val_score(svm, X, y, cv=10, scoring='accuracy').mean())

# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X,y)
print('KNN:', cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())


Logistic Regression: 0.768937728938
Decision Tree: 0.8263003663
Random Forest: 0.864212454212
SVM: 0.614578754579
Naive Bayes: 0.550256410256
KNN: 0.768937728938


In [49]:
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('train_reduced_CME_test.csv',delimiter=',')
# print(train.shape)

# get rid of first row (I think this is just column labels)
train = train[1:,:]

# replace missing featureVals with 0.5 (halfway in between)
where_are_NaNs = np.isnan(train)
train[where_are_NaNs] = 0.5

y = train[:,4] # label
X = train[:,0:3] # data

In [57]:
from sklearn.cross_validation import cross_val_score

#TODO use F-1 for scoring, or a hybrid if possible

# logistic regression
from sklearn.linear_model import LogisticRegression
logregFillNA = LogisticRegression()
logregFillNA.fit(X,y)
print('Logistic Regression:', cross_val_score(logregFillNA, X, y, cv=10, scoring='accuracy').mean())

# decision tree
from sklearn.tree import DecisionTreeClassifier
dtFillNA = DecisionTreeClassifier()
dtFillNA.fit(X,y)
print('Decision Tree:', cross_val_score(dtFillNA, X, y, cv=10, scoring='accuracy').mean())

# random forest
from sklearn.ensemble import RandomForestClassifier
forestFillNA = RandomForestClassifier(n_estimators=10, max_depth=10)
forestFillNA.fit(X,y)
print('Random Forest:', cross_val_score(forestFillNA, X, y, cv=10, scoring='accuracy').mean())

# support vector machine
from sklearn.svm import LinearSVC
svmFillNA = LinearSVC()
svmFillNA.fit(X,y)
print('SVM:', cross_val_score(svmFillNA, X, y, cv=10, scoring='accuracy').mean())

# naive bayes
from sklearn.naive_bayes import GaussianNB
gnbFillNA = GaussianNB()
gnbFillNA.fit(X,y)
print('Naive Bayes:', cross_val_score(gnbFillNA, X, y, cv=10, scoring='accuracy').mean())

# KNN
from sklearn.neighbors import KNeighborsClassifier
knnFillNA = KNeighborsClassifier(n_neighbors=20)
knnFillNA.fit(X,y)
print('KNN:', cross_val_score(knnFillNA, X, y, cv=10, scoring='accuracy').mean())


Logistic Regression: 0.868144499179
Decision Tree: 0.882334428024
Random Forest: 0.904018427294
SVM: 0.502681992337
Naive Bayes: 0.868144499179
KNN: 0.868144499179


Logistic Regression: 0.0
Decision Tree: 0.885905856596
Random Forest: 0.907343550447
SVM: 0.568746579091
Naive Bayes: 0.868144499179
KNN: 0.868144499179


/Users/tammi/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/tammi/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/tammi/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/tammi/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/tammi/anaconda/lib/python3.5/site-packages/sklearn/metric